# 👶 Baby Crib Monitoring Demo using VideoDB RTStream
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16Kp95r7Mp6HkjwILeFABit5s4erAphX6?usp=sharing)
## 📖 Storytime: Why This Matters

Meet **Vidit** and **Meghna** — a young couple juggling demanding jobs and household responsibilities. After a long, exhausting day, all they hope for is a peaceful night’s sleep. But their energetic little one has other plans.

Their child, once safely tucked into his crib, has recently discovered how to climb out. While the parents sleep, unaware, the baby risks injury by wandering unsupervised at night. How can they keep him safe without losing their much-needed rest?

---

## 🎥 Enter VideoDB RTStream

**VideoDB** offers the perfect solution for this problem. Using **RTStream**, we can let AI continuously monitor a live video feed, index scenes, detect specific events like **baby attempting to climb out of the crib**, and instantly send alerts to the parents when something risky happens.

In this demo, **Vidit and Meghna install an IP camera near the crib** and use **VideoDB RTStream** to power an AI monitoring system.  
As soon as the baby makes a move to climb out, AI detects it, triggers an event, and fires a real-time alert so the parents can step in.

---

## 🚀 What You’ll Build in This Notebook

By the end of this notebook, you’ll learn how to:
- Connect a live RTSP video stream to VideoDB
- Continuously analyze video scenes using AI-generated natural language descriptions
- Detect specific events like *“baby escaping crib”*
- Trigger real-time alerts on such events

So — **do you want to build an intelligent AI-powered baby monitor too?**  
Let’s get started!


---
## 📦 Step 1: Install Dependencies  
Before setting up the AI-powered baby monitor, let’s install the necessary VideoDB SDK.


In [32]:
!pip install -q videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Let's connect to VideoDB's API using your credentials to prepare for stream monitoring.

Please enter your `VIDEO_DB_API_KEY` in the input box that appears below after you run this cell.

Your input will be masked.


In [34]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---

## 📦 Step 3: Connect to the Baby Crib RTSP Stream  
We’ll now connect to the live video stream of the crib using its RTSP URL.

In this demo, the stream is running at `rtsp://samples.rts.videodb.io:8554/crib`.




In [35]:
rtsp_url = "rtsp://samples.rts.videodb.io:8554/crib"
crib_stream = coll.connect_rtstream(
    name="Baby Crib Monitor",
    url=rtsp_url,
    sample_rate=30
)
print(crib_stream)

RTStream(id=rts-019711a0-0fde-7911-b282-25bc0b4ecf65, name=Baby Crib Monitor, collection_id=None, created_at=None, sample_rate=30, status=connected)



#### If you have already connected the stream, run the below cell with the **rtstream id** to reconnect.

In [39]:
# crib_stream = coll.get_rtstream("")

In [56]:
# To stop the stream
# crib_stream.stop()

In [57]:
# To start the stream
# crib_stream.start()

---
### 👀 Let us have a look at the crib stream


#### 📺 Helper Function: Display Video Stream

This cell contains a small utility function to help visualize the video streams with helpful information. You don't need to modify this code.

In [43]:
# To display the stream with relevant information

from IPython.display import HTML
import re
from datetime import datetime

def display_clean_stream(video_url, video_name="🎥 Camera Feed"):
    match = re.search(r'/(\d{16})-(\d{16})\.m3u8', video_url)
    if match:
        start_ts = int(match.group(1)) / 1e6
        end_ts = int(match.group(2)) / 1e6
        start_time = datetime.utcfromtimestamp(start_ts).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.utcfromtimestamp(end_ts).strftime('%Y-%m-%d %H:%M:%S')
        time_range = f"{start_time} → {end_time} UTC"
    else:
        time_range = "Time Unknown"

    return HTML(f"""
    <div style="position:relative;width:640px;">
      <video id="hls-video" width="640" height="360" controls muted autoplay style="border-radius:6px;"></video>
      <div style="position:absolute;top:10px;left:10px;background:rgba(0,0,0,0.6);color:#fff;padding:6px 12px;border-radius:4px;font-size:13px;font-family:sans-serif;">
        <strong>{video_name}</strong><br>{time_range}
      </div>
      <script src="https://cdn.jsdelivr.net/npm/hls.js@latest"></script>
      <script>
        var video=document.getElementById('hls-video');
        if(Hls.isSupported()){{
          var hls=new Hls();
          hls.loadSource("{video_url}");
          hls.attachMedia(video);
        }}else if(video.canPlayType('application/vnd.apple.mpegurl')){{
          video.src="{video_url}";
        }}
      </script>
    </div>
    """)



#### 🔗 Get & Display Recent Stream

This cell uses the helper function above to fetch and display the last few minutes of the stream.

In [46]:
# To get last few minutes stream link
import time

def play_stream(rtstream):

    now = int(time.time())
    start = int(now - (5 * 60))
    stream_url = rtstream.generate_stream(start, now)
    return stream_url

video_url = play_stream(crib_stream)

video_name = "👶 Baby Monitor · Crib Activity Feed"
display_clean_stream(video_url, video_name)

---

## 📦 Step 4: Index Scenes with Descriptions  
Now, we'll create a real-time scene index that periodically analyzes the video and generates natural language descriptions of what’s happening in the crib.

The AI model will watch for activity such as the baby moving, sitting, or attempting to climb out.


In [47]:
from videodb import SceneExtractionType

crib_scene_index = crib_stream.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={
        "time": 10,
        "frame_count": 1,
    },
    prompt="Describe the activity of the baby kept inside a baby crib. Notice if baby climbs out or attempts to escape.",
    name="Baby_Crib_Index",
)
crib_index_id = crib_scene_index.rtstream_index_id
print("Scene Index ID:", crib_index_id)

Scene Index ID: 5722fbf82669a81e



#### If you have already created a scene index, run the below cell with your **scene index id** to reconnect.

In [48]:
# crib_index_id = ""
# crib_scene_index = crib_stream.get_scene_index(crib_index_id)

In [ ]:
# To stop the index
# crib_scene_index.stop()

In [ ]:
# To start the index
# crib_scene_index.start()

---
### Let us see the result of the scene indexing

In [68]:
def display_scenes(rtstream_scene_index):
    """
    Displays descriptions of the latest 5 scenes from a given crib index.
    """
    scenes = rtstream_scene_index.get_scenes(page_size=5)

    if scenes and scenes.get("scenes"):
        for scene in scenes.get("scenes"):
            description = scene["description"]
            print(description)
            print("-" * 80)
    else:
        print("Scenes not found for given index.")

display_scenes(crib_scene_index)

The baby is outside the crib, standing on the floor. The baby is wearing a light blue pajama set. The baby is not attempting to climb out of the crib, as they are already out. The baby is standing near the crib, facing away from it.
--------------------------------------------------------------------------------
The baby is actively trying to climb out of the crib. The baby is positioned with its torso over the top rail of the crib, attempting to lower itself down. The baby is wearing a light blue onesie.
--------------------------------------------------------------------------------
The baby is actively trying to climb out of the baby crib. The baby has managed to get one arm and part of their torso over the side of the crib, indicating a clear attempt to escape.
--------------------------------------------------------------------------------
The baby is actively climbing on the side of the crib, attempting to climb out. The baby is standing on a colorful blanket or bedding that is n

---
## 📦 Step 5: Define an Event for Baby Escape  
We’ll now create an event in VideoDB to detect when the AI spots the baby attempting to climb out.


In [69]:
event_id = conn.create_event(
    event_prompt="Detect if the baby is trying to escape or climbing out of the crib.",
    label="baby_escape",
)
print("Event ID:", event_id)


Event ID: 3adc40d26d6fed0d


---
## 📦 Step 6: Attach an Alert for the Escape Event  
To complete our setup, we’ll link a real-time alert to this event, which will notify the parents instantly through a webhook.


In [70]:
alert_id = crib_scene_index.create_alert(
    event_id,
    callback_url="https://ec3897ac1f70252c839eb4a20c4114c3.m.pipedream.net"
)
print("Alert ID:", alert_id)

Alert ID: 06792489c7386e07


---
## 📡 Example Alert Received at the Webhook

Here’s a real-time alert sent by our AI monitoring system to the webhook URL when the baby was detected trying to climb out of the crib:

> **Webhook URL:** `https://ec3897ac1f70252c839eb4a20c4114c3.m.pipedream.net/`

---

**📬 Incoming Alert Payload (body):**

```json
{
  "event_id": "event-3adc40d26d6fed0d",
  "label": "baby_escape",
  "confidence": 0.95,
  "explanation": "The baby is actively climbing out of the crib, posing a potential safety risk.",
  "timestamp": "2025-05-27T12:44:30.916875+00:00",
  "start_time": "2025-05-27T18:14:27.080270+05:30",
  "end_time": "2025-05-27T18:14:37.080270+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019711a0-0fde-7911-b282-25bc0b4ecf65/1748349867000000-1748349877000000.m3u8"
}
```

---

### 📌 What This Tells Us:
- **event_id** — Unique ID for the event trigger  
- **label** — Type of event detected (`baby_escape`)  
- **confidence** — AI confidence score (0.95 = 95% sure)  
- **explanation** — AI-generated description of what’s happening  
- **timestamp** — When the alert was created  
- **start_time / end_time** — Time range of the detected scene  
- **stream_url** — This is a temporary link you can use to view the detected scene

✅ This confirms our AI-powered baby monitor is working perfectly — detecting escape attempts and instantly sending alerts.


---
### Now we can stop the stream.

In [31]:
crib_stream.stop()

---

## 🌙 Wrapping Up: Peace of Mind for Vidit and Meghna

In this notebook, we built a smart, AI-powered video monitoring system for a real-life problem faced by Vidit and Meghna.  

With this system in place, Vidit and Meghna can finally sleep peacefully, knowing their child is being safely monitored through AI-driven surveillance.  


---


**But this is just one story.**

- What if the same system could monitor an elderly parent at home — detecting falls or prolonged inactivity?  
- Or watch over a pet while the family is away, alerting them if it leaves a safe zone or behaves unusually?  
- Or even notify parents when a toddler approaches dangerous areas like staircases or kitchen counters?

The possibilities of real-time video intelligence at home are endless.

**What would *you* monitor next?**